In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import re
import requests
import holoviews as hv
import matplotlib.pyplot as plt
import pint
import nd2reader
import tifffile
import scipy.stats
import skimage

In [ ]:
%load_ext autoreload
%autoreload 2
import simulation

In [ ]:
# u = simulation.ureg
class ObjProxy(object):
    def __init__(self, module_name, attr_name):
        self.__module_name = module_name
        self.__attr_name = attr_name

    def __getattr__(self, name):
        return getattr(getattr(globals()[self.__module_name], self.__attr_name), name)


u = ObjProxy("simulation", "ureg")

In [ ]:
hv.extension("bokeh")

# Setup

In [ ]:
excitation_bins = np.linspace(300, 850, 1000)
emission_bins = excitation_bins

# FP spectra

In [ ]:
# list of urls -> dict of spectra (columns: ex, em, 2p)
# rebin spectra
# rebinned spectra -> 2D spectra (see-saw)

# 2D spectrum * excitation spectrum * QY [ex] -> [em]

# rebin laser spectrum
# sample line * rebinned laser spectrum = excitation image [ex,y,x]
# excitation image (above) -> emission ground truth [em,y,x]
# emission ground truth + reflectivity * excitation = emission [em,y,x]

In [ ]:
spectra_urls = [
    "https://www.fpbase.org/spectra_csv/?q=79,80,158",
    "https://www.fpbase.org/spectra_csv/?q=1451,1450",
    "https://www.fpbase.org/spectra_csv/?q=119,120",
    "https://www.fpbase.org/spectra_csv/?q=121,122",
]

In [ ]:
spectra = simulation.import_fpbase_spectra(spectra_urls)

In [ ]:
(-(x - 1) * (x + 1) * x).max()

In [ ]:
x = np.linspace(-1, 1, 100)
plt.plot(x, -(x - 1) * (x + 1) * x * 3 * np.sqrt(3) / 2)

In [ ]:
def valid_range(ary):
    is_valid = ~np.isnan(ary)
    idx1 = is_valid.argmax()
    idx2 = len(ary) - is_valid[::-1].argmax()
    return idx1, idx2


def seesaw_spectrum(spectrum, amount):
    x = np.zeros(len(spectrum))
    idx1, idx2 = valid_range(spectrum)
    x[idx1:idx2] = np.linspace(-1, 1, idx2 - idx1)
    x = x.reshape((-1,) + (1,) * (np.ndim(spectrum) - 1))
    shape = -(x - 1) * (x + 1) * x * 3 * np.sqrt(3) / 4
    new_spectrum = (amount * shape + 1 / 2) * spectrum
    new_spectrum /= np.nanmax(new_spectrum, axis=0)
    return new_spectrum

In [ ]:
a = seesaw_spectrum(
    spectra["mCherry2"]["em"].values[:, np.newaxis], np.array([0.3, 0.5])[np.newaxis, :]
)
plt.plot(a)

In [ ]:
a = seesaw_spectrum(
    spectra["mCherry2"]["em"].values[:, np.newaxis],
    np.linspace(-0.5, 1, len(excitation_bins))[np.newaxis, :],
)
plt.imshow(a)

In [ ]:
plt.plot(seesaw_spectrum(spectra["mCherry2"]["em"], 1))
plt.plot(seesaw_spectrum(spectra["mCherry2"]["em"], -1))
plt.plot(seesaw_spectrum(spectra["mCherry2"]["em"], 0.2))
plt.plot(seesaw_spectrum(spectra["mCherry2"]["em"], -0.2))

In [ ]:
old_index = spectra["mCherry2"].index.astype(np.float)
excitation_index = pd.Index(excitation_bins, name="wavelength")
new_index = excitation_index.union(old_index)
b = spectra["mCherry2"].reindex(index=new_index)
b.interpolate(method="nearest", inplace=True)

In [ ]:
b.loc[excitation_index].plot()

In [ ]:
excitation_index

In [ ]:
b.plot()

In [ ]:
np.nanmax(a["ex"])

# Sample

In [ ]:
# nd2 = nd2reader.ND2Reader("ClpP_mEGFP_100x.nd2")
# nd2._parser._raw_metadata.z_levels = [0]
# sample_img = nd2.get_frame_2D(v=0, c=1)  # [:,500:1000]
# sample_img = image_to_xarray(sample_img, nd2.metadata["pixel_microns"])

In [ ]:
sample_img = tifffile.imread("t000001xy14c2.tif")
scale = 4.25 / 20
sample_img = simulation.image_to_xarray(sample_img, scale)

In [ ]:
sample_img.plot.imshow()

In [ ]:
import skimage.filters

In [ ]:
labels = sample_img.values > skimage.filters.threshold_otsu(sample_img.values)

In [ ]:
num_spectra = 4

In [ ]:
spectra_idx = skimage.morphology.label(labels) % num_spectra + (labels != 0)

# Excitation line

In [ ]:
line_img = simulation.draw_excitation_line(
    np.linspace(0.6, 1.3, len(excitation_bins)) * u.mm,
    0.3 * u.um,
    0 * u.um,
    1,
    p_vertical=3,
    p_horizontal=6,
    height_px=60,
    width_px=200,
)

In [ ]:
line_img.nbytes / 1e6

In [ ]:
line_img[0].plot()

In [ ]:
line_img[-1].plot()

In [ ]:
sample_offset = simulation.offset_xarray(sample_img, line_img, dict(x=500, y=470))

In [ ]:
(sample_offset * line_img).plot(aspect=20, size=5)